In [9]:
import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests
import warnings

warnings.filterwarnings('ignore')

### Pinpointed location of site 

In [10]:
sg_lat = 40.66563889
sg_lon = -73.5700833 

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False,
)

# Put a marker at the stream gauge location
folium.Marker(
    [sg_lat, sg_lon], popup="Stream Gage in Long Island, New York"
).add_to(m)
m 



In [11]:
nwis_url = (
    "https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no="
    "01310500&legacy=&referred_module=sw&period=&begin_date=2001-09-01&end_"
    "date=2023-09-01 "
)

# Send an HTTP GET request to the URL
nwis_response = requests.get(nwis_url)
nwis_response.raise_for_status()



In [12]:
ny_q_df = pd.read_csv(
    BytesIO(nwis_response.content),
    comment="#",
    delimiter="\t",
    skiprows=[29, 30],
    names=["agency_cd", "site_no", "datetime", "streamflow_cfs", "code"],
    index_col="datetime",
    parse_dates=True,
    na_values="Ice",
)


In [13]:


ny_flood_df = ny_q_df["2005-09":"2022-09"]




In [14]:


ny_ann_max_q_df = ny_q_df[['streamflow_cfs']].resample('AS').max()




### Maximum streamflow in 2005 with highs in 2014 in Long Island, 

The graph shows streamflows in Long Island, New York. In the graph we can see spikes in 2005, 2010, 2014, and in 2021. These spikes are due to various floods or hurricanes. We can also see a gap in data on the graph from January of 2010 to January of 2012 which could possibly be due to Hurricane Sandy which hit the area around that time.  


In [15]:
ny_time_plot = ny_ann_max_q_df.hvplot(
    x='datetime', y='streamflow_cfs', 
    title='Streamflows in New York',
    xlabel='Year', ylabel='Streamflow in cubic feet per second'
)

ny_time_plot



:Curve   [datetime]   (streamflow_cfs)

In [16]:
%%capture
%%bash
jupyter nbconvert ny-timeseries.ipynb --to html --no-input